In [14]:
import os

In [15]:
parent_dir = "../data"

In [16]:

all_files = []
for root, dirs, files in os.walk(parent_dir):
    for file in files:
        if file.endswith((".gitkeep")):
            continue
        all_files.append(os.path.join(root, file))


In [17]:
from markitdown import MarkItDown
from openaivec import pandas_ext
from openai import AzureOpenAI, AsyncAzureOpenAI
import pandas as pd

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    base_url=os.getenv("AZURE_OPENAI_BASE_URL"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

async_client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    base_url=os.getenv("AZURE_OPENAI_BASE_URL"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

md = MarkItDown(llm_client=client, llm_model="gpt-5-mini")

pandas_ext.set_client(client)
pandas_ext.set_async_client(async_client)
pandas_ext.set_responses_model("gpt-5-mini")


client.responses.create(model="gpt-5-mini", input="Hello, world!", reasoning={"effort": "minimal"}).output_text



'Hello! How can I help you today?'

In [19]:
sources = pd.DataFrame({"file_path": all_files})

In [20]:
docs = (
    sources
    .assign(
        parsed=lambda df: df.file_path
        .map(md.convert)
        .map(lambda x: x.text_content)
    )
)

In [23]:
for file_path, parsed in zip(docs.file_path, docs.parsed):
    with open(f"{str(file_path)}.md".replace("data", "parsed"), "w") as f:
        try:
            f.write(parsed)
        except:
            f.write("Error in writing parsed content.")

In [25]:
from pydantic import BaseModel, Field

class Question(BaseModel):
    question: str = Field(
        ...,
        description=(
            "生成AIに対する入力プロンプトとして使用される質問文。"
            "この質問文単体で完全に意味が成立し、外部の文脈（セクションのタイトル、セクション本文、"
            "ドキュメント全体の内容、周囲の文章）に依存してはならない。"
            "指示語（「これ」「それ」「あれ」「この機能」「このプログラム」「上記」など）は厳禁。"
            "対象となる用語・固有名詞・機能名・手順名・制度名などは質問文内に明示的に含めること。"
            "読者が質問文だけを読んで何について尋ねられているか100%理解できるように作成する。"
            "FAQ の一般的な問いの形式（例：『〜とは何ですか？』『〜を行うにはどうすればよいですか？』など）を用いる。"
            "質問文を作成する前に必ず以下の内部手順を実行する："
            "1) セクション内容から固有名詞・概念・機能・要件・手順などの重要要素を抽出する。"
            "2) 抽出した要素を明示的に含めた、外部文脈不要の質問文を構成する。"
            "3) 質問文単体で意味が完全に通じるか検証し、不足があれば固有名詞を補完する。"
            "4) 他の質問と内容が重複しないようにする。"
        )
    )
    answer: str = Field(
        ...,
        description=(
            "質問文に対する回答文。質問に直接かつ明確に答えることを最優先とし、"
            "質問文に含まれる情報の範囲内で説明を行う。"
            "外部文脈に依存した説明や脱線は避け、必要であれば補足情報を加えてもよいが、"
            "質問文の主題に対する回答が常に中心となるようにする。"
        )
    )

class Section(BaseModel):
    title: str = Field(..., description="セクションのタイトル。")
    content: str = Field(..., description="セクションの本文。FAQ生成の情報源となる。")
    faqs: list[Question] = Field(
        ...,
        description=(
            "このセクションに基づいて生成された FAQ のリスト。"
            "1 セクションにつき少なくとも 2 件以上の FAQ を生成すること。"
            "同じ意味や内容を言い換えただけの重複質問は作らないこと。"
        )
    )

class Document(BaseModel):
    sections: list[Section] = Field(..., description="ドキュメント全体のセクション一覧。")

In [ ]:
instructions = """
あなたは提供されたドキュメントから高品質な FAQ を生成する専門アシスタントです。
以下の Pydantic モデルに完全準拠した JSON を生成してください。

{
  "sections": [
    {
      "title": "string",
      "content": "string",
      "faqs": [
        {
          "question": "string",
          "answer": "string"
        }
      ]
    }
  ]
}

## FAQ 生成ルール（必ず遵守すること）

### 【最重要】question は生成AIに単独で与えるプロンプトである
- question は他の情報（セクションタイトル、セクション本文、文脈）を一切参照できない。
- question 単体だけで意味が完全に成立していなければならない。
- question 文字列だけで「何について尋ねているか」が100%理解できる必要がある。

### 1. 質問文の具体的要件
- 指示語（「これ」「それ」「あれ」「この機能」「上記」等）禁止
- 名称・対象・仕組み・プロセスなどは質問文内に必ず固有名詞として記述
- 文脈補完を必要とする省略・曖昧表現は禁止
- FAQ の一般的な形式で記述する（例：「〜とは何ですか？」「〜の手順は？」）

### 2. 内部手順（生成時に必ず行う）
1. セクション本文から固有名詞・機能・手順・要件を抽出  
2. それらを質問文に明示的に含める  
3. 質問文単体で意味が完結していることを確認  
4. 他の質問と重複していないことを確認  

### 3. 回答文の要件
- 質問内容に直接・具体的に答える
- 外部文脈なしに理解できるように説明する
- 不要な脱線は禁止

### 4. FAQ 数
- 各セクションにつき最低 2 件以上，出来るだけ多く生成する
- 含まれる情報についてはMECEに質問文を作成する

## 出力形式
- 出力は **JSON のみ**
- JSON 以外の文章は禁止
- 情報ソースが英語でも必ず日本語の FAQ を生成すること
- 上記 Pydantic モデルの構造に完全準拠すること

以上のルールに基づき、FAQ を生成してください。
"""

docs["document"] = await (
    docs.parsed.aio.responses(
        instructions=instructions,
        response_format=Document,
        # reasoning={"effort": "minimal"},
        batch_size=1,
        max_concurrency=5,
    )
)

Processing batches:   0%|          | 0/4 [00:00<?, ?item/s]

In [32]:
(
    docs
    .assign(
        file_path=lambda df: df.file_path.map(lambda x: x.split("/")[-1])
    )
    .drop(columns=["parsed"])
    .ai.extract("document")
    .explode("document_sections")
    .ai.extract("document_sections")
    .explode("document_sections_faqs")
    .ai.extract("document_sections_faqs")
    .reset_index(drop=True)
    .to_csv("../parsed/faqs.csv", index=False)
)